In [1]:
import os
import time


import numpy as np

import cv2
from PIL import Image
import gc
import torch

from my_utlis.annotator import MyAnnotator
from my_utlis.models import CarClassifier, CustomResnext, DtpClassifier
# Id Видео с дтп
dtp_videos = [
    2,
    4,
    5,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    21,
    22,
    24,
    27,
    33,
    36,
    50
]


def main(car_classifier_weights, yolo_weights, yolo_conf, video_dir):
    # Загружаем yolo
    model = torch.hub.load('ultralytics/yolov5', 'custom',
                           path=yolo_weights)
    model.conf = 0.5
    # C:\Users\igors/.cache\torch\hub\ultralytics_yolov5_master
    # C:\\Users\\igors/.cache\\torch\\hub\\ultralytics_yolov5_master
    # model = torch.hub.load('ultralytics/yolov5', 'custom', 'weights/yolo_weights.pt')
    # model.conf = args.yolo_conf
    gc.collect()
    # Модель для классификации типа машин
    car_classfier = CarClassifier(car_classifier_weights)

    # dtp_classifier = DtpClassifier('../weights/trans_224.pth')
    acc = 0
    all_dtp_cnt = 0
    for video_number, video_file in enumerate(os.listdir(video_dir)):

        cap = cv2.VideoCapture(os.path.join('../data/video', video_file))
        video_id = video_file.split('.')[0]
        cnt_dtp = 0
        cnt_not_dtp = 0
        num_frame = 0
        obr_frame = 0
        while cap.isOpened():
            # читаем кадр из видео
            ret, frame = cap.read()
            t = time.time()
            num_frame += 1
            with_dtp = False
            if num_frame % 30 != 0:
                continue
            if ret is True and frame.size != 0:
                obr_frame += 1
                frame_copy = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                results = model(result, size=1280)
                dets = []
                if not results.pandas().xyxy[0].empty:
                    for res in range(len(results.pandas().xyxy[0])):
                        r = results.pandas().xyxy[0].to_numpy()[res]
                        # print(r)

                        dets.append(list(r))

                np_dets = np.array(dets)
                # 
                for i in range(len(dets)):
                    x0, y0, x1, y1, conf, cls_id, cls_name = dets[i]
                    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
                    # MyAnnotator.rectangle(frame, (x0, y0), (x1, y1))
                    # MyAnnotator.put_text(frame,
                    #                      cls_name,
                    #                      (x0, y1))
                    if cls_name not in ['car', 'truck', 'bus', 'boat', 'motorcycle']:
                        continue
                    # MyAnnotator.put_text(frame, cls_name + " " + str(round(conf, 2)), (x0, y0))
                    
                    # Классификация типа машин
                    crop = frame_copy[y0:y1, x0:x1]
                    car_name, conf_cls, np_pred = car_classfier.infer(crop)
                    # Если была хотя бы одна спец техника в кадре, то кадр с подозрением на дтп
                    if car_name != 'car':
                        with_dtp = True
                    else:
                        cnt_not_dtp += 1
                    # print(num_frame)
                if with_dtp:
                    cnt_dtp += 1
            else:
                break
        # Если число кадров с дтп больше 7 или составляет хотя бы треть от общего числа кадров,
        # то ставим видео класс дтп
        if cnt_dtp >= 7 or cnt_dtp >= 1/3 * obr_frame:
            pred = 'dtp'
        else:
            pred = 'not_dtp'
        if pred == 'dtp':
            all_dtp_cnt += 1
        if int(video_id) in dtp_videos:
            true_label = 'dtp'
        else:
            true_label = 'not_dtp'

        acc += int(true_label == pred)
        print('video number:', video_number + 1, '/', len(os.listdir('../data/video')))
        print('video id:', video_id)
        print('current_accuracy:', round(acc / (video_number + 1), 2))
        print('true label: ', true_label)
        print('pred label:', pred)
        print('Count frame with dtp', cnt_dtp)
        print('All frame count', obr_frame)
        print('---------------------------')

    print('Final accuracy:', round(acc / 50, 2))

if __name__ == "__main__":
    # Path to dtp classifier weights
    car_classifier_weights = '../weights/eff_model_car.pth'
    # Path to yolo weights
    yolo_weights = '../weights/yolov5m6.pt'
    # Yolo conf
    yolo_conf = 0.5
    # Path to directory with video
    video_dir = '../data/video'
    main(car_classifier_weights, yolo_weights, yolo_conf, video_dir)


Using cache found in C:\Users\igors/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-11-22 torch 1.10.0 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)

Fusing layers... 
Model Summary: 378 layers, 35704908 parameters, 0 gradients
Adding AutoShape... 


video number: 1 / 50
video id: 1
current_accuracy: 1.0
true label:  not_dtp
pred label: not_dtp
Count frame with dtp 0
All frame count 20
---------------------------
video number: 2 / 50
video id: 10
current_accuracy: 1.0
true label:  dtp
pred label: dtp
Count frame with dtp 34
All frame count 59
---------------------------
video number: 3 / 50
video id: 11
current_accuracy: 1.0
true label:  dtp
pred label: dtp
Count frame with dtp 10
All frame count 38
---------------------------
video number: 4 / 50
video id: 12
current_accuracy: 1.0
true label:  dtp
pred label: dtp
Count frame with dtp 10
All frame count 39
---------------------------
video number: 5 / 50
video id: 13
current_accuracy: 1.0
true label:  dtp
pred label: dtp
Count frame with dtp 11
All frame count 31
---------------------------
video number: 6 / 50
video id: 14
current_accuracy: 0.83
true label:  dtp
pred label: not_dtp
Count frame with dtp 0
All frame count 33
---------------------------
video number: 7 / 50
video id: